# CNN for Cifar-10 Dataset

## Requirements

For this project we are used:
* Python 3.6
* *Keras* from Tensorflow 1.8.0
* *GPUs parallel calculation manager* nVidia CUDA 9.0
* *GPU-accelerated library* nVidia cuDNN 7.1

## Introduction

In this first notebook we are going to explain how we have set up our work.  
Essentially the process is divided into several parts:
* Set up environment with the Cifar-10 Dataset
* Define a convolutional neural network
* Define a quantization method
* Train the convolutional neural network
* Provide information about CNN's performance and accuracy

## Cifar-10 Dataset
Cifar-10 Dataset is taken from the official website www.cs.toronto.edu.

Dataset is stored in the data directory: cnn/data. From Cifar-10 dataset we are going to take x_train, t_train, x_test and t_test.
The training dataset set is used for training the CNN, the testing dataset is used for evaluate the performance and the accuracy of the network.

In [1]:
import numpy as np
import tensorflow as tf

/home/daibak/.virtualenvs/aca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# First sketch

In [2]:
"""This module provides utilities to download, init and load cifar10 dataset."""

from pathlib import Path

import h5py
from tensorflow import keras

In [3]:
DATA_URL = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
DATA_DIR = 'cnn/data'
DATASET_FILE = 'cifar10.h5'


def load_dataset_as_tensors():
    """Load dataset from h5 (not lazily)."""

    h5_filename = Path(DATA_DIR) / DATASET_FILE
    with h5py.File(h5_filename.absolute(), 'r') as ds:
        return (ds['data/train'][()], ds['label/train'][()],
                ds['data/test'][()], ds['label/test'][()])


def preprocess_dataset(x_train, t_train, x_test, t_test, NCHW=False):
    """Preprocess dataset: label -> one hot encodeing, transpose if not in NCHW order."""

    t_train = keras.utils.to_categorical(t_train).astype(np.uint8)
    t_test = keras.utils.to_categorical(t_test).astype(np.uint8)

    if not NCHW:
        return (x_train.transpose(0, 2, 3, 1), t_train,
                x_test.transpose(0, 2, 3, 1), t_test)
    else:
        return (x_train, t_train, x_test, t_test)

In [4]:
x_train, t_train, x_test, t_test = load_dataset_as_tensors()

In [5]:
x_train.shape, t_train.shape, x_test.shape, t_test.shape

((50000, 3, 32, 32), (50000,), (10000, 3, 32, 32), (10000,))

Uniform data and visualize shape

In [6]:
x_train, t_train, x_test, t_test = preprocess_dataset(x_train, t_train, x_test,
                                                      t_test)

In [7]:
x_train.shape, t_train.shape, x_test.shape, t_test.shape

((50000, 32, 32, 3), (50000, 10), (10000, 32, 32, 3), (10000, 10))

## CNN Model and Training

This CNN is called *dense_cnn*. Here we will explain how it is composed.

The CNN is composed by several layers. In the first part there are 2 **convolutional** layers and 2 **pooling** layers (they are alternated), then there are a *flatten* layer followed by a **relu** layer, a *dropout* layer and finally a **softmax** layer.

The network uses a stochastic gradient descent optimizer and a categorical crossentropy loss.  
To judge the performance of our model we are used a MSE metric.

In [49]:
N_TRAIN_SAMPLES = x_train.shape[0]
N_TEST_SAMPLES = x_test.shape[0]
BATCH_SIZE = 64

model = keras.Sequential(
    name='dense_cnn',
    layers=[
        keras.layers.Conv2D(
            32,
            5,
            input_shape=(32, 32, 3),
            padding='same',
            activation=tf.nn.relu),
        keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
        keras.layers.Conv2D(64, 5, padding='same', activation=tf.nn.relu),
        keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
        keras.layers.Flatten(),
        keras.layers.Dense(1024, activation=tf.nn.relu),
        keras.layers.Dropout(0.4),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

model.compile(
    loss='categorical_crossentropy',
    optimizer='sgd',
    metrics=['accuracy', keras.metrics.mean_squared_error])

This network is trained for 50 epochs.

In [50]:
hist = model.fit(
    x=x_train,
    y=t_train,
    batch_size=BATCH_SIZE,
    epochs=50,
    validation_split=0.2,
    shuffle=False,
    initial_epoch=0)

Train on 40000 samples, validate on 10000 samples
Epoch 3/3
40000/40000 [==============================] - 139s 3ms/step - loss: 14.4815 - acc: 0.1014 - mean_squared_error: 0.1797 - val_loss: 14.5917 - val_acc: 0.0947 - val_mean_squared_error: 0.1811


In [51]:
hist.history

{'val_loss': [14.591711572265625],
 'val_acc': [0.0947],
 'val_mean_squared_error': [0.1810600138902664],
 'loss': [14.481510899353028],
 'acc': [0.10145],
 'mean_squared_error': [0.1797071960926056]}

# With graph

In [94]:
class SoftmaxClassifier:
    def __init__(self,
                 name,
                 forward_pass_fn,
                 loss_fn,
                 eval_fn,
                 optimizer=tf.train.GradientDescentOptimizer(0.001),
                 fake_quantization=False):
        self.name = name
        self.forward_pass = forward_pass
        self.loss_fn = loss_fn
        self.eval_fn = eval_fn
        self.optimizer = optimizer
        self.fake_quantization = fake_quantization
        train_ops_graph = self.train_graph()
        eval_ops_graph = self.evaluate_graph()
        predict_ops_graph = self.predict_graph()
        self.tb_path = "/tmp/log-tb/" + self.name + "/"
        self.save_path = "/tmp/" + self.name + ".ckpt"

    def infer(self):

        self._training_placeholder = tf.placeholder(tf.bool, shape=())

        logits = self.forward_pass(
            train_mode_placeholder=self._training_placeholder)

        predictions = {
            "logits": logits,
            "classes": tf.argmax(logits, axis=1, name="classes"),
            "probabilities": tf.nn.softmax(logits, name="softmax")
        }

        return predictions

    def calculate_loss(self, logits):
        loss = self.loss_fn(logits=logits)
        tf.summary.scalar("loss", loss)
        return loss

    def optimize(self, loss):
        return self.optimizer.minimize(loss)

    def evaluate_op(self, predictions):
        return eval_fn(predictions)

    def train_graph(self):
        graph = tf.Graph()

        with graph.as_default() as g:
            predictions = infer()
            loss = calculate_loss(predictions["logits"])
            train_op = optimize()
            evals = evaluate_op(predictions)
            summaries = tf.summary.merge_all()

        ops = predictions
        ops.update(evals)
        ops["loss"] = loss
        ops["summaries"] = summaries
        ops["train_op"] = train_op

        return ops, graph

    def evaluate_graph(self):
        graph = tf.Graph()

        with graph.as_default() as g:
            predictions = infer()
            loss = calculate_loss(predictions["logits"], labels)
            evals = evaluate_op(predictions)
            summaries = tf.summary.merge_all()

        ops = predictions
        ops.update(evals)
        ops["loss"] = loss
        ops["summaries"] = summaries

        return ops, graph

    def predict_graph(self):
        graph = tf.Graph()

        with graph.as_default() as g:
            predictions = infer()

        return predictions, graph

    def _split_and_batch(inputs,
                         input_names,
                         batch_size,
                         validation_split):

        n_samples = inputs[0].shape
        input_tensors = list(map(tf.get_default_graph().get_tensor_by_name,
                            input_names))
        input_DL = dict(zip(input_tensors, inputs))  # Dict from pairs of list

        for k, v in input_DL.items():
            input_DL[k] = np.split(v, [round((1 - validation_split) * n_samples)])

        input_LD = [dict(zip(input_DL, t))
                    for t in zip(*input_DL.values())]  #List of Dicts

        train_dict = input_LD[0]
        val_dict = input_LD[1]

        n_train_samples = train_dict[input_tensors[0]].shape[0]
        n_batches, drop = np.divmod(n_train_samples, batch_size)

        for k, v in train_dict.items():
            train_dict[k] = np.array_split(v, n_batches)

        train_LD = [dict(zip(train_dict, t)) for t in zip(*train_dict.values())]

        val_dict.update({"training_placeholder": False})
        mode_t = {"training_placeholder": True}
        for d in input_LD:
            d.update(mode_t)

        return train_LD, val_dict
    
    def fit(self,
            inputs,
            input_names=["features", "labels"],
            batch_size=1,
            validation_split=0,
            epochs=1,
            verbosity=0):

        ops, graph = self.train_ops_graph
        saver = tf.train.Saver()
        history = []

        with tf.Session(graph=graph) as sess:
            sess.run(tf.global_variables_initializer())
            train_LD, val_dict = self._split_and_batch(
                inputs, input_names, batch_size, validation_split)

            val_dict.update({self._training_placeholder: False})

            if verbose >= 1:
                summary_writer = tf.summary.FileWriter(self.tb_path,
                                                       sess.graph)
                print("Launch: tensorboard --logdir=" + self.tb_path)
                i = 1

            for e in range(1, epochs + 1):
                sess.run(tf.local_variables_initializer())

                for train_dict in train_LD:
                    out = sess.run(ops, feed_dict=train_dict)
                    if verbosity >= 1:
                        summary_writer.add_summary(out["summaries"], i)
                        summary_writer.flush()
                        i = i + 1
                    if verbosity == 2:
                        print(out)

                out = sess.run(
                    {x: ops[x]
                     for x in ops if x not in ["train_op"]},
                    feed_dict=val_dict)

                history.append(out)

            saver.save(sess, self.save_path)

        return dict(
            zip(history[0],
                zip(*[out.values() for out in history])))  #Dict of lists

    def predict(self, inputs, input_names=["features"]):
        ops, graph = self.predict_ops_graph
        saver = tf.train.Saver()

        input_tensors = map(tf.get_default_graph().get_tensor_by_name,
                            input_names)
        input_dict = dict(zip(input_tensors, inputs))
        input_dict.update({self._training_placeholder: False})

        with tf.Session(graph=graph) as sess:
            sess.run(tf.global_variables_initializer())
            saver.restore(sess, self.save_path)

            out = sess.run(ops, feed_dict=input_dict)

        return out

    def evaluate(self, inputs, input_name=["features", "labels"]):
        ops, graph = self.eval_ops_graph
        saver = tf.train.Saver()

        input_tensors = map(tf.get_default_graph().get_tensor_by_name,
                            input_names)
        input_dict = dict(zip(input_tensors, inputs))
        input_dict.update({self._training_placeholder: False})

        with tf.Session(graph=graph) as sess:
            sess.run(tf.global_variables_initializer())
            saver.restore(sess, self.save_path)

            out = sess.run(ops, feed_dict=input_dict)

        return out

    def freeze():
        pass

    def transform():
        pass

In [108]:
def dense_froward_pass(train_mode_placeholder=None):
    features = tf.placeholder(
        tf.uint8, shape=(None, 32, 32, 3), name="features")

    conv1 = tf.layers.conv2d(
        inputs=inputs,
        filters=32,
        kernel_size=5,
        padding="same",
        activation=tf.nn.relu)

    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=(2, 2), strides=2)

    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=5,
        padding="same",
        activation=tf.nn.relu)

    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=(2, 2), strides=2)

    pool2_flat = tf.layers.flatten(pool2)

    dense = tf.layers.dense(
        inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=train_mode_placeholder)

    return tf.layers.dense(inputs=dropout, units=10, name="logits")

In [54]:
def loss_fn(logits):
    labels = tf.placeholder(tf.uint8, [None, 10], name="labels")
    return tf.losses.sparse_softmax_cross_entropy(
        labels=labels, logits=logits), labels

In [55]:
def eval_fn(predictions):
    labels = tf.get_default_graph().get_tensor_by_name("labels:0")
    eval_metrics = {
        "accuracy":
        tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]),
        "mse":
        tf.metrics.mean_squared_error(
            labels=labels, predictions=predictions["logits"])
    }
    return eval_metrics

In [99]:
g = tf.Graph()

with g.as_default():
    tf.

In [ ]:
inputs = [x_train, t_train]
input_names = ["features", "labels"]
batch_size = 64
validation_split = 0.2

n_samples = inputs[0].shape[0]
input_tensors = input_names
input_DL = dict(zip(input_tensors, inputs))  # Dict from pairs of list

for k, v in input_DL.items():
    input_DL[k] = np.split(v, [round((1 - validation_split) * n_samples)])

input_LD = [dict(zip(input_DL, t))
            for t in zip(*input_DL.values())]  #List of Dicts

train_dict = input_LD[0]
val_dict = input_LD[1]

n_train_samples = train_dict[input_tensors[0]].shape[0]
n_batches, drop = np.divmod(n_train_samples, batch_size)

for k, v in train_dict.items():
    train_dict[k] = np.array_split(v, n_batches)

train_LD = [dict(zip(train_dict, t)) for t in zip(*train_dict.values())]

val_dict.update({"training_placeholder": False})
mode_t = {"training_placeholder": True}
for d in input_LD:
    d.update(mode_t)

In [115]:
train_LD[0]["features"].shape

(64, 32, 32, 3)

In [ ]:
tf.layers.conv2d